<a href="https://colab.research.google.com/github/arrpitk/product-review-sentiment-analyzer/blob/main/notebooks/sentiment_analysis_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from datasets import load_dataset

# Load Yelp reviews
dataset = load_dataset("yelp_review_full")
print(dataset)
# Map 1-2 stars as negative, 3 as neutral, 4-5 as positive
def preprocess_function(examples):
    examples["label"] = [0 if star <= 2 else 1 if star >= 4 else 2 for star in examples["label"]]
    return examples

processed_dataset = dataset.map(preprocess_function,batched=True)
train_dataset = processed_dataset["train"].shuffle(seed=42).select(range(10000))
eval_dataset = processed_dataset["test"].shuffle(seed=42).select(range(1000))

README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

# Train the model
trainer.train()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arrpitk (arrpitk-coforge) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.599000,0.472126
2,0.460700,0.489337
3,0.364800,0.484662


TrainOutput(global_step=1875, training_loss=0.4452015584309896, metrics={'train_runtime': 1477.984, 'train_samples_per_second': 20.298, 'train_steps_per_second': 1.269, 'total_flos': 3974092830720000.0, 'train_loss': 0.4452015584309896, 'epoch': 3.0})

In [13]:
import evaluate
import numpy as np

# Load accuracy metric
metric = evaluate.load("accuracy")

# Define evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Evaluate
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Epoch,Training Loss,Validation Loss
0,No log,0.500016


In [5]:
# Save model locally
model_path = "./finetuned_sentiment_model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

# Optional: Download to local machine
from google.colab import files
!zip -r finetuned_sentiment_model.zip {model_path}
files.download("finetuned_sentiment_model.zip")

  adding: finetuned_sentiment_model/ (stored 0%)
  adding: finetuned_sentiment_model/training_args.bin (deflated 51%)
  adding: finetuned_sentiment_model/model.safetensors (deflated 8%)
  adding: finetuned_sentiment_model/tokenizer_config.json (deflated 75%)
  adding: finetuned_sentiment_model/config.json (deflated 49%)
  adding: finetuned_sentiment_model/special_tokens_map.json (deflated 42%)
  adding: finetuned_sentiment_model/tokenizer.json (deflated 71%)
  adding: finetuned_sentiment_model/vocab.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
model_name = "product-review-sentiment-analyzer"
# Configure the model to be pushed to Hub
model_name = "product-review-sentiment-analyzer"
trainer.args.hub_model_id = f"arpitk/{model_name}"
trainer.args.push_to_hub = True

# Train the model


# The model will be pushed automatically after training
# or you can manually push it
trainer.push_to_hub(commit_message="Add fine-tuned sentiment model")

events.out.tfevents.1742212721.8a1641e842e1.611.0:   0%|          | 0.00/6.87k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1742214386.8a1641e842e1.611.1:   0%|          | 0.00/10.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/arpitk/results/commit/6854bc5da38c0e989e1476326abd6ae78ab51350', commit_message='Add fine-tuned sentiment model', commit_description='', oid='6854bc5da38c0e989e1476326abd6ae78ab51350', pr_url=None, repo_url=RepoUrl('https://huggingface.co/arpitk/results', endpoint='https://huggingface.co', repo_type='model', repo_id='arpitk/results'), pr_revision=None, pr_num=None)